In [106]:
from lxml import etree
import requests
import lxml.html

In [107]:
# get the url and parse the link
url = "https://www.cancer.gov/about-cancer/treatment/types/targeted-therapies/targeted-therapies-fact-sheet"
rp = requests.get(url)
page = lxml.html.fromstring(rp.text)
#page = etree.HTML(rp.text)

In [108]:
therapies = page.xpath("//section[h2[@id = 'what-targeted-therapies-have-been-approved-for-specific-types-of-cancer']]//p//a/@href")
therapies

['/about-cancer/treatment/drugs/atezolizumab',
 '/about-cancer/treatment/drugs/nivolumab',
 '/about-cancer/treatment/drugs/durvalumab',
 '/about-cancer/treatment/drugs/avelumab',
 '/about-cancer/treatment/drugs/pembrolizumab',
 '/about-cancer/treatment/drugs/erdafitinib',
 'https://www.cancer.gov/about-cancer/treatment/drugs/enfortumabvedotin-ejfv',
 '/about-cancer/treatment/drugs/bevacizumab',
 '/about-cancer/treatment/drugs/everolimus',
 '/about-cancer/treatment/drugs/everolimus',
 '/about-cancer/treatment/drugs/tamoxifencitrate',
 '/about-cancer/treatment/drugs/toremifene',
 '/about-cancer/treatment/drugs/trastuzumab',
 '/about-cancer/treatment/drugs/fulvestrant',
 '/about-cancer/treatment/drugs/anastrozole',
 '/about-cancer/treatment/drugs/exemestane',
 '/about-cancer/treatment/drugs/lapatinibditosylate',
 '/about-cancer/treatment/drugs/letrozole',
 '/about-cancer/treatment/drugs/pertuzumab',
 '/about-cancer/treatment/drugs/ado-trastuzumab-emtansine',
 '/about-cancer/treatment/drug

In [380]:
# test if the url is exist
link = "https://www.cancer.gov/about-cancer/treatment/drugs/nivolumab"
try:
    response = requests.get(link)
    print("URL is valid and exists on the internet")
except requests.ConnectionError as exception:
    print("URL does not exist on Internet")

URL is valid and exists on the internet


In [381]:
# get the targeted therapy url and parse it
ther_url = requests.get(link) 
#ther_page = etree.HTML(ther_url.text)
ther_page = lxml.html.fromstring(ther_url.content)

In [382]:
ther_name = ther_page.xpath("//article/div/h1/text()")
ther_name

['Nivolumab']

In [383]:
# get dailyMed link
dailyMedURL = ther_page.xpath("//article//div/p/a[text() = 'FDA label information for this drug is available at DailyMed.']/@href")
dailyMedURL

['https://dailymed.nlm.nih.gov/dailymed/drugInfo.cfm?setid=f570b9c4-6846-4de2-abfa-4d0a4ae4e394&audience=consumer']

In [384]:
# extract the corresponding diseases (only for bold text, but non-bold is left (could be improved))
#diseases = ther_page.xpath("//article//div[h2[contains(text(),'Use in Cancer')]]/ul/li/strong//text()")
diseases = ther_page.xpath("//article//div[h2[contains(text(),'Use in Cancer')]]/ul/li/strong")
for disease in diseases:
    #print(desease)
    print(disease.text_content())

Classic Hodgkin lymphoma
Colorectal cancer
Hepatocellular carcinoma
Malignant pleural mesothelioma.
Melanoma.
Non-small cell lung cancer.
Renal cell carcinoma
Squamous cell carcinoma of the esophagus
Squamous cell carcinoma of the head and neck
Urothelial carcinoma


In [385]:
# remove punctuation at the end of disease
punc = '''!()-[]{};:'"\, <>./?@#$%^&*_~'''

In [401]:
diseaseList = []
for disease in diseases[1]:
    dis = str(disease.text_content())
    if dis[-1] in punc:  
        dis = dis.replace(dis[-1], "")  
    print(dis)
    diseaseList.append(dis)
    path = "//article//div[h2[contains(text(),'Use in Cancer')]]/ul/li[strong/*[contains(text(),'" + dis + "')]]"
    diseaseContent = ther_page.xpath(path)
    print(len(diseaseContent))
    if len(diseaseContent)>0:

        print(diseaseContent[0].text_content())

Colorectal cancer
1
Colorectal cancer in adults and children 12 years and older. It is used alone or with ipilimumab to treat metastatic microsatellite instability-high (MSI-H) or mismatch repair deficient (dMMR) cancer that got worse after treatment with a fluoropyrimidine, oxaliplatin, and irinotecan hydrochloride.¹


In [404]:
# do NER
geneProtein = ''
text = diseaseContent[0].text_content()

doc = nlp_craft(str(text))
# filter a certain type
for entity in doc.ents:
    if entity.label_=='GGP':
        geneProtein = geneProtein + ', ' + entity.text

doc = nlp_jnlpha(str(text))
# filter a certain type
for entity in doc.ents:
    if entity.label_=='PROTEIN':
        geneProtein = geneProtein + ', ' + entity.text

print(geneProtein[2:])

irinotecan hydrochloride.¹


In [189]:
'a' + 'b'

'ab'

# Bio-NER using scispacy

In [5]:
!pip install -U spacy

Requirement already up-to-date: spacy in /Users/damon/.pyenv/versions/3.7.0/lib/python3.7/site-packages (2.3.5)
You should consider upgrading via the '/Users/damon/.pyenv/versions/3.7.0/bin/python3.7 -m pip install --upgrade pip' command.


In [6]:
!pip install scispacy

  Using cached scispacy-0.3.0-py3-none-any.whl (42 kB)
  Using cached pysbd-0.3.3-py3-none-any.whl (67 kB)
  Using cached nmslib-2.0.11-cp37-cp37m-macosx_10_14_x86_64.whl (919 kB)
  Using cached pybind11-2.6.2-py2.py3-none-any.whl (191 kB)
     |████████████████████████████████| 236 kB 3.6 MB/s eta 0:00:01
You should consider upgrading via the '/Users/damon/.pyenv/versions/3.7.0/bin/python3.7 -m pip install --upgrade pip' command.


In [28]:
import os
os.getcwd()

'/Users/damon/Desktop/Georgetown University/Georgetown U/Research Project'

In [1]:
# install the pre-tained model
# online
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.3.0/en_ner_craft_md-0.3.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.3.0/en_ner_jnlpba_md-0.3.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.3.0/en_ner_bc5cdr_md-0.3.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.3.0/en_ner_bionlp13cg_md-0.3.0.tar.gz
# local
#!pip install /Users/damon/Desktop/Georgetown\ University/Georgetown\ U/Research\ Project/en_core_sci_md-0.3.0.tar.gz

  Using cached https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.3.0/en_ner_craft_md-0.3.0.tar.gz (79.8 MB)
  Using cached https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.3.0/en_ner_jnlpba_md-0.3.0.tar.gz (79.8 MB)
  Using cached https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.3.0/en_ner_bc5cdr_md-0.3.0.tar.gz (79.8 MB)


  Using cached https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.3.0/en_ner_bionlp13cg_md-0.3.0.tar.gz (79.8 MB)


In [2]:
# import necessary packages
import scispacy
import spacy
# the model we are going to use
#import en_core_web_sm
from spacy import displacy
from scispacy.abbreviation import AbbreviationDetector
from scispacy.umls_linking import UmlsEntityLinker

In [3]:
# Define the pre-trained model.
nlp_craft = spacy.load("/Users/damon/.pyenv/versions/3.7.0/lib/python3.7/site-packages/en_ner_craft_md/en_ner_craft_md-0.3.0") 
nlp_jnlpha = spacy.load("/Users/damon/.pyenv/versions/3.7.0/lib/python3.7/site-packages/en_ner_jnlpba_md/en_ner_jnlpba_md-0.3.0") 
nlp_bc5cdr = spacy.load("/Users/damon/.pyenv/versions/3.7.0/lib/python3.7/site-packages/en_ner_bc5cdr_md/en_ner_bc5cdr_md-0.3.0") 
nlp_bionlp13cg = spacy.load("/Users/damon/.pyenv/versions/3.7.0/lib/python3.7/site-packages/en_ner_bionlp13cg_md/en_ner_bionlp13cg_md-0.3.0") 

In [434]:
text = '''Colorectal cancer in adults and children 12 years and older. It is used alone or with ipilimumab to treat metastatic microsatellite instability-high (MSI-H) or mismatch repair deficient (dMMR) cancer that got worse after treatment with a fluoropyrimidine, oxaliplatin, and irinotecan hydrochloride.¹'''

In [435]:
doc = nlp_craft(str(text))
print(list(doc.sents))

[Colorectal cancer in adults and children 12 years and older., It is used alone or with ipilimumab to treat metastatic microsatellite instability-high (MSI-H) or mismatch repair deficient (dMMR) cancer that got worse after treatment with a fluoropyrimidine, oxaliplatin, and irinotecan hydrochloride.¹]


In [436]:
# Find the Bio-Medical Entities in the given text
print(doc.ents)

(hydrochloride.¹,)


In [413]:
# visualize dependency parses
from spacy import displacy
displacy.render(next(doc.sents), style='dep', jupyter=True)

In [414]:
# visualize the entities
# displacy.serve(doc, style="ent")

In [415]:
for entity in doc.ents:
    print(entity.text, entity.label_)

irinotecan hydrochloride.¹ PROTEIN


In [416]:
# filter a certain type
for i in doc.ents:
    if i.label_=='GGP':
        print(i.text)